# Analysing International Debt

<p>
  Debt isn't only a tool used by individuals to cover personal expenses; countries also take on debt to manage their economies.
  For example, a significant portion of a country's budget may be spent on infrastructure, which is crucial for ensuring a good
  quality of life for its citizens. The World Bank is one of the main organizations that provides loans to countries for such purposes.
</p>

<p>
  In this analysis, we will explore a dataset from The World Bank that tracks international debt. This dataset includes information
  on the amount of debt (in USD) owed by developing countries across different categories. Our goal is to answer questions such as:
</p>

<ul>
  <li>What is the total amount of debt owed by the countries listed in the dataset?</li>
  <li>Which country owes the most debt, and how much is it?</li>
  <li>What is the average debt amount across different debt categories for all countries?</li>
  <li>What are the distinct debt indicators in the dataset?</li>
  <li>What is the highest amount of principal repayment?</li>
  <li>What is the most common debt indicator?</li>
  <li>Are there any other notable debt-related trends or issues?</li>
</ul><br>

<div style="text-align: center;">
  <img src="https://epicforamerica.org/wp-content/uploads/2025/10/muska-create-ujDa5lOt_P4-unsplash-scaled.jpg" alt="Debt Data Visualization" style="width: 60%; max-width: 600px;">
</div>

---

## 1. The World Bank's International Debt Data
To get started, we’ll connect to the <code>international_debt</code> database, where the table containing this data is stored.  
First, we’ll run a simple query to <code>SELECT</code> all columns from the <code>international_debt</code> table, but we’ll limit the output to just the first ten rows to keep things manageable.


In [55]:
%%sql
postgresql:///international_debt

    SELECT *
    FROM international_debt
    LIMIT 10
    

10 rows affected.


country_name,country_code,indicator_name,indicator_code,debt
Afghanistan,AFG,"Disbursements on external debt, long-term (DIS, current US$)",DT.DIS.DLXF.CD,72894453.700000003
Afghanistan,AFG,"Interest payments on external debt, long-term (INT, current US$)",DT.INT.DLXF.CD,53239440.100000001
Afghanistan,AFG,"PPG, bilateral (AMT, current US$)",DT.AMT.BLAT.CD,61739336.899999999
Afghanistan,AFG,"PPG, bilateral (DIS, current US$)",DT.DIS.BLAT.CD,49114729.399999999
Afghanistan,AFG,"PPG, bilateral (INT, current US$)",DT.INT.BLAT.CD,39903620.100000001
Afghanistan,AFG,"PPG, multilateral (AMT, current US$)",DT.AMT.MLAT.CD,39107845
Afghanistan,AFG,"PPG, multilateral (DIS, current US$)",DT.DIS.MLAT.CD,23779724.300000001
Afghanistan,AFG,"PPG, multilateral (INT, current US$)",DT.INT.MLAT.CD,13335820
Afghanistan,AFG,"PPG, official creditors (AMT, current US$)",DT.AMT.OFFT.CD,100847181.900000006
Afghanistan,AFG,"PPG, official creditors (DIS, current US$)",DT.DIS.OFFT.CD,72894453.700000003


---

## 2. Finding the number of distinct countries
<p> In the first ten rows, we can see how much debt <em>Afghanistan</em> owes in different debt categories. However, we don’t know how many unique countries are in the dataset yet. The same country may appear multiple times because it has debt in more than one category. </p> <p> To do our analysis properly, we need to know how many unique countries are listed. In this section, we will find out the number of distinct countries in the dataset. </p>

In [57]:
%%sql
SELECT 
    COUNT(DISTINCT country_name) AS total_distinct_countries
FROM international_debt;

 * postgresql:///international_debt
1 rows affected.


total_distinct_countries
124


---

## 3. Finding out the distinct debt indicators
<p> In the dataset, we have 124 countries listed. To better understand the types of debt these countries have, we need to look at two key columns: </p> <ul> <li><strong><code>indicator_name</code></strong>: This column provides a brief description of why a country took on debt. For example, it could describe whether the debt was taken for infrastructure, education, or health.</li> <li><strong><code>indicator_code</code></strong>: This column indicates the specific category or type of debt. The code represents different categories, such as external debt, domestic debt, or loans for specific purposes (e.g., principal repayment, interest payments, etc.).</li> </ul> <p> Understanding these debt indicators is important for a few reasons: </p> <ul> <li><strong>Identifying Debt Purposes:</strong> By knowing what each debt indicator represents, we can understand why countries are borrowing money and for what purposes (e.g., to build infrastructure or pay interest).</li> <li><strong>Classifying Debt:</strong> The <code>indicator_code</code> helps us categorize the debt. For example, some countries might have high debt in the "external debt" category, while others may have debt related to "domestic investments."</li> <li><strong>Analyzing Debt Trends:</strong> By examining the different debt indicators, we can observe patterns or trends, such as which types of debt are most common or which countries are relying heavily on specific categories of debt.</li> </ul>​

In [59]:
%%sql
SELECT DISTINCT indicator_code AS distinct_debt_indicators
FROM international_debt
ORDER BY distinct_debt_indicators

 * postgresql:///international_debt
25 rows affected.


distinct_debt_indicators
DT.AMT.BLAT.CD
DT.AMT.DLXF.CD
DT.AMT.DPNG.CD
DT.AMT.MLAT.CD
DT.AMT.OFFT.CD
DT.AMT.PBND.CD
DT.AMT.PCBK.CD
DT.AMT.PROP.CD
DT.AMT.PRVT.CD
DT.DIS.BLAT.CD


---

## 4. Totaling the amount of debt owed by the countries
<p> A country's financial debt is often seen as a reflection of its economic health. However, if we look at this from a global perspective, how do we assess the total debt owed by all countries combined? </p> <p> To understand the broader picture of global debt, we need to shift our focus from individual debt indicators to the total amount of debt owed by each country. Here's how we can approach this: </p> <ul> <li><strong>Total Debt Calculation:</strong> By adding up all the debt amounts (in USD) across all countries, we can get a sense of the global debt burden. This total will help us understand the overall financial obligations of countries around the world.</li> <li><strong>Insight into Global Economic Health:</strong> The total debt owed by countries can provide a snapshot of the global economy. High total debt might indicate that many countries are heavily indebted, which could be a sign of economic instability. On the other hand, lower total debt could suggest healthier economic conditions.</li> <li><strong>Comparing Debt Across Countries:</strong> By totaling the debt for each country, we can compare how different nations are managing their debt. This can help us identify countries with the highest debt burdens and understand the disparities between richer and poorer nations.</li> </ul> <p> In this section, we will calculate the total debt owed by all the countries in the dataset. This will give us a clearer picture of how the world’s economies are performing and how much financial pressure countries are facing as a whole. </p>

In [61]:
%%sql
SELECT 
    ROUND(SUM (debt) / 1000000, 2) AS total_debt
FROM international_debt; 

 * postgresql:///international_debt
1 rows affected.


total_debt
3079734.49


---

## 5. Country with the highest debt
<p>Very large numbers are hard for people to understand. When we talk about more than <em>3 million <strong>million</strong></em> USD in debt, it’s difficult to really imagine how big that amount is.</p> <p>Now that we have calculated the total debt for different countries, we can find out which country has the highest overall debt and the exact amount. <br><br><strong>Note</strong> that this number is the total of several types of debt added together. Looking at the total helps give a general idea of a country’s economic situation. We could also check which category makes up most of this debt, but we will not cover that here.</p>

In [63]:
%%sql
SELECT 
    country_name, 
    SUM(debt) AS total_debt
FROM international_debt
GROUP BY country_name
ORDER BY total_debt DESC
LIMIT 1;

 * postgresql:///international_debt
1 rows affected.


country_name,total_debt
China,285793494734.200001568


---

## 6. Average amount of debt across indicators
<p>The country with the highest total debt is <em>China</em>. A more detailed breakdown of China’s debt is available <a href="https://datatopics.worldbank.org/debt/ids/country/CHN">here</a>.</p> <p>At this point, we have a basic understanding of the dataset and some summary statistics. We also know the different debt indicators used to describe how countries owe money. Next, we can look at the average amount of debt across these indicators. This will help us better understand how debt is spread across different categories.</p>

In [65]:
%%sql
SELECT 
    indicator_code AS debt_indicator,
    indicator_name,
    AVG(debt) AS average_debt
FROM international_debt
GROUP BY debt_indicator, indicator_name
ORDER BY average_debt DESC
LIMIT 10;

 * postgresql:///international_debt
10 rows affected.


debt_indicator,indicator_name,average_debt
DT.AMT.DLXF.CD,"Principal repayments on external debt, long-term (AMT, current US$)",5904868401.499193612
DT.AMT.DPNG.CD,"Principal repayments on external debt, private nonguaranteed (PNG) (AMT, current US$)",5161194333.812658349
DT.DIS.DLXF.CD,"Disbursements on external debt, long-term (DIS, current US$)",2152041216.890243888
DT.DIS.OFFT.CD,"PPG, official creditors (DIS, current US$)",1958983452.859836046
DT.AMT.PRVT.CD,"PPG, private creditors (AMT, current US$)",1803694101.963265321
DT.INT.DLXF.CD,"Interest payments on external debt, long-term (INT, current US$)",1644024067.650806481
DT.DIS.BLAT.CD,"PPG, bilateral (DIS, current US$)",1223139290.398230108
DT.INT.DPNG.CD,"Interest payments on external debt, private nonguaranteed (PNG) (INT, current US$)",1220410844.421518983
DT.AMT.OFFT.CD,"PPG, official creditors (AMT, current US$)",1191187963.083064523
DT.AMT.PBND.CD,"PPG, bonds (AMT, current US$)",1082623947.653623188


---

## 7. The highest amount of principal repayments
<p>The indicator <code>DT.AMT.DLXF.CD</code> has the highest average debt. This category covers repayments of long-term debt. Countries often take on long-term debt to get funding when they need it. More details about this indicator can be found <a href="https://datacatalog.worldbank.org/principal-repayments-external-debt-long-term-amt-current-us-0">here</a>.</p> <p>One thing to notice is the large gap between the top two indicators and the rest. This suggests that these two categories make up the biggest portions of debt for many countries.</p> <p>To understand this better, we can look at which country has the highest debt in long-term repayments (<code>DT.AMT.DLXF.CD</code>). Since countries face different economic challenges, this can give us a clearer picture of that country’s financial situation.</p>

In [67]:
%%sql
SELECT 
    country_name, 
    indicator_name
FROM international_debt
WHERE debt = (SELECT 
                 MAX(debt)
             FROM international_debt
             WHERE indicator_code = 'DT.AMT.DLXF.CD');

 * postgresql:///international_debt
1 rows affected.


country_name,indicator_name
China,"Principal repayments on external debt, long-term (AMT, current US$)"


---

## 8. The most common debt indicator
<p>China has the highest amount of debt in the long-term debt category (<code>DT.AMT.DLXF.CD</code>). This matches the data published by <a href="https://data.worldbank.org/indicator/DT.AMT.DLXF.CD?end=2018&most_recent_value_desc=true">the World Bank</a>. Checking results against trusted sources helps confirm that our analysis is correct.</p> <p>We found that long-term debt has the highest average amount. The next step is to see whether it is also the most common type of debt across countries.</p>

In [69]:
%%sql
SELECT indicator_code, COUNT(indicator_code) AS indicator_count
FROM international_debt
GROUP BY indicator_code
ORDER BY indicator_count DESC, indicator_code DESC
LIMIT 20

 * postgresql:///international_debt
20 rows affected.


indicator_code,indicator_count
DT.INT.OFFT.CD,124
DT.INT.MLAT.CD,124
DT.INT.DLXF.CD,124
DT.AMT.OFFT.CD,124
DT.AMT.MLAT.CD,124
DT.AMT.DLXF.CD,124
DT.DIS.DLXF.CD,123
DT.INT.BLAT.CD,122
DT.DIS.OFFT.CD,122
DT.AMT.BLAT.CD,122


---

## 9. Other viable debt issues and conclusion
<p>There are six debt indicators that appear for all countries in our dataset, and <code>DT.AMT.DLXF.CD</code> is one of them. This suggests that these countries share some common economic challenges. However, this is only part of the overall picture.</p> <p>Now, instead of focusing on debt indicators, we will shift back to the debt amounts. We will look at the highest amount of debt for each country. This can help highlight other possible economic issues a country may be facing.</p> <p>In this notebook, we explored the debt owed by countries around the world. We reviewed summary statistics, identified several key patterns, and checked our results to ensure the analysis was accurate.</p>

In [71]:
%%sql
SELECT country_name,
MAX(debt) AS maximum_debt
FROM international_debt
GROUP BY country_name
ORDER BY maximum_debt DESC
LIMIT 10

 * postgresql:///international_debt
10 rows affected.


country_name,maximum_debt
China,96218620835.699996948
Brazil,90041840304.100006104
Russian Federation,66589761833.5
Turkey,51555031005.800003052
South Asia,48756295898.199996948
Least developed countries: UN classification,40160766261.599998474
IDA only,34531188113.199996948
India,31923507000.799999237
Indonesia,30916112653.799999237
Kazakhstan,27482093686.400001526


## Conclusion

<ul>
  <li>The dataset contains international debt information from the World Bank for developing countries across multiple debt categories.</li>
  <li>The total amount of debt owed by all countries in the dataset is extremely large, highlighting the scale of global borrowing.</li>
  <li>China is the country with the highest overall debt among those listed in the dataset.</li>
  <li>The average debt amount varies across indicators, with long-term debt having the highest average value.</li>
  <li>The dataset includes several distinct debt indicators that represent different types of borrowing and repayments.</li>
  <li>Long-term debt repayment (<code>DT.AMT.DLXF.CD</code>) has the highest principal repayment amount.</li>
  <li>Long-term debt is also one of the most common debt indicators across countries, appearing for all countries in the dataset.</li>
  <li>Overall, the analysis shows that long-term borrowing is a major economic factor for many developing nations and highlights shared debt-related challenges.</li>
</ul>
